In [1]:
import torch
import numpy as np
from torch.autograd import Variable
import matplotlib.pyplot as plt
import torchvision
import torch.nn as nn

In [33]:
torch.set_default_tensor_type('torch.FloatTensor')

In [34]:
def squash(s, dim=-1, epsilon=1e-3):
#     print(s.size())
    norm_s = torch.sqrt(torch.norm(s) + epsilon)
#     print(norm_s)
    v = (norm_s / (1 + norm_s)) * (s / norm_s)
    return v

In [35]:
# https://discuss.pytorch.org/t/why-softmax-function-cant-specify-the-dimension-to-operate/2637
def softmax(input, axis=1):
    input_size = input.size()
    
    trans_input = input.transpose(axis, len(input_size)-1)
    trans_size = trans_input.size()

    input_2d = trans_input.contiguous().view(-1, trans_size[-1])
    
    soft_max_2d = nn.functional.softmax(input_2d)
    
    soft_max_nd = soft_max_2d.view(*trans_size)
    return soft_max_nd.transpose(axis, len(input_size)-1)

In [190]:
class CapsuleLayer(nn.Module):
    def __init__(self, in_channels, capsule_dimension, num_capsules, kernel_size=None, routing=False, num_iterations=0, stride=1):
        
        super(CapsuleLayer, self).__init__()
        
        self.in_channels = in_channels
        self.capsule_dimension = capsule_dimension
        self.num_capsules = num_capsules
        self.kernel_size = kernel_size
        self.routing = routing
        self.num_iterations = num_iterations
        self.stride = stride
        
        if not self.routing:
            
            self.conv = nn.Conv2d(self.in_channels, self.capsule_dimension * self.num_capsules,\
                                  self.kernel_size, self.stride)      
            
        
        else:
            
            # todo: to be calculated later
            self.width = 1152
            
            self.weights = torch.autograd.Variable(torch.from_numpy(0.01 * np.random.randn(1, self.width,\
                                                                                           self.num_capsules, self.capsule_dimension,\
                                                                                           self.in_channels)), requires_grad=True)
            
            

        
    def forward(self, x):
        
        if not self.routing:
            
            self.conv_out = self.conv(x)
            
            # todo: to be calculated later
            width = 6 # (W - F + 2 * P) / S + 1
            
            conv2 = self.conv_out.view(-1, width * width * self.num_capsules, self.capsule_dimension)
            
            squash_conv = squash(conv2)
            
            return squash_conv
                
                
        else:
            # x -> [batch size , 1152, 8]
             
            batch_size = x.size()[0]
            
            print("x", x.size())
            print("w", self.weights.size())
            
            u = x.unsqueeze(2).expand(x.size()[0], x.size()[1], self.num_capsules, x.size()[2]).unsqueeze(4).type(torch.FloatTensor)
            
            self.weights = self.weights.expand(batch_size, self.weights.size()[1], self.weights.size()[2],\
                                              self.weights.size()[3], self.weights.size()[4]).type(torch.FloatTensor)
    
            print("w with batch", self.weights.size())
            print("u", u.size())
            
            print("w type", type(self.weights))
            print("u type", type(u))
                    
            u_hat = torch.matmul(self.weights, u)
        
            print("u_hat", u_hat.size())
            
            v = None
            
            for r in range(self.num_iterations):
                
                b = torch.zeros(batch_size, x.size()[1], self.num_capsules, 1, 1).type(torch.FloatTensor)
                
                c = softmax(b, axis=2)
                
                c = c.expand(c.size()[0], c.size()[1], c.size()[2], u_hat.size()[3], c.size()[4])
                
                print("c", c.size())
                
#                 print("c numpy", c.data.cpu().numpy().shape)
#                 print("u_hat numpy", u_hat.data.cpu().numpy().shape)

#                 s_numpy = np.matrix.dot(c.data.cpu().numpy(), u_hat.data.cpu().numpy())
                
#                 print("s_numpy", s_numpy.shape)
                
                s = torch.sum(c * u_hat, dim=2)
                
#                 print("s", s.size(), s)
                
                v = squash(s, dim=-2)
                
                print("v", v.size())
            
                b += torch.mm(u_hat, v)
                
                print("b", b.size())
                
            return v
                
            
        

In [191]:
class Net(nn.Module):
    def __init__(self):
        
        super(Net, self).__init__()
        
        self.conv1 = nn.Conv2d(1, 256, 9)
        
        self.primary_capsule = CapsuleLayer(256, 8, 32, kernel_size=9, routing=False, num_iterations=0, stride=2)
        
        self.digital_capsule = CapsuleLayer(8, 16, 10, kernel_size=None, routing=True, num_iterations=10, stride=1)
        
        self.criterion = torch.nn.MSELoss(size_average=True)
        
        
    def forward(self, x, target):
        
        x = self.conv1(x)
        
        x = nn.functional.relu(x)
        
        x = self.primary_capsule(x)
        
        x = self.digital_capsule(x)
        
        _, indices = torch.max(x, axis=2)
        
        y_predicted = torch.squeeze(indices)
                
        loss = self.ceriation(y_predicted, target)
        
        return x, loss
        

In [192]:
net = Net()

In [193]:
net

Net (
  (conv1): Conv2d(1, 256, kernel_size=(9, 9), stride=(1, 1))
  (primary_capsule): CapsuleLayer (
    (conv): Conv2d(256, 256, kernel_size=(9, 9), stride=(2, 2))
  )
  (digital_capsule): CapsuleLayer (
  )
  (criterion): MSELoss (
  )
)

In [87]:
import torchvision.datasets as dset
import torchvision.transforms as transforms
## load mnist dataset
root = '../data'
download = False
trans = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (1.0,))])
train_set = dset.MNIST(root=root, train=True, transform=trans, download=download)
test_set = dset.MNIST(root=root, train=False, transform=trans)

batch_size = 128
# kwargs = {'num_workers': 1}
train_loader = torch.utils.data.DataLoader(
                 dataset=train_set,
                 batch_size=batch_size,
                 shuffle=True)
test_loader = torch.utils.data.DataLoader(
                dataset=test_set,
                batch_size=batch_size,
                shuffle=False)


In [88]:
import torch.optim as optim
import torch.nn.functional as F

In [194]:
optimizer = optim.Adam(net.parameters(), lr=0.01)
for epoch in range(10):
    # trainning
    for batch_idx, (x, target) in enumerate(train_loader):
        optimizer.zero_grad()
        x, target = Variable(x), Variable(target)
        _, loss = net(x, target)
        loss.backward()
        optimizer.step()
        if batch_idx % 100 == 0:
            print('==>>> epoch: {}, batch index: {}, train loss: {:.6f}'.format(epoch, batch_idx, loss.data[0]))
#     # testing
#     correct_cnt, ave_loss = 0, 0
#     for batch_idx, (x, target) in enumerate(test_loader):
#         x, target = Variable(x, volatile=True), Variable(target, volatile=True)
#         score, loss = net(x, target)
#         _, pred_label = torch.max(score.data, 1)
#         correct_cnt += (pred_label == target.data).sum()
#         ave_loss += loss.data[0]
#     accuracy = correct_cnt*1.0/len(test_loader)/batch_size
#     ave_loss /= len(test_loader)
#     print('==>>> epoch: {}, test loss: {:.6f}, accuracy: {:.4f}'.format(epoch, ave_loss, accuracy))

torch.save(net.state_dict(), 'capsule_net')

x torch.Size([128, 1152, 8])
w torch.Size([1, 1152, 10, 16, 8])
w with batch torch.Size([128, 1152, 10, 16, 8])
u torch.Size([128, 1152, 10, 8, 1])
w type <class 'torch.autograd.variable.Variable'>
u type <class 'torch.autograd.variable.Variable'>
u_hat torch.Size([128, 1152, 10, 16, 1])
c torch.Size([128, 1152, 10, 16, 1])
v torch.Size([128, 1152, 16, 1])


RuntimeError: matrices expected, got 5D, 4D tensors at /Users/soumith/miniconda2/conda-bld/pytorch_1503975474428/work/torch/lib/TH/generic/THTensorMath.c:1288